# PrioritizeMe

Aplikacja do priorytetowego planowania zadań to narzędzie, które pomoże efektywnie zarządzać czasem i zadaniami. Dzięki niej możesz tworzyć zadania, przypisywać im tytuły oraz opisy, a następnie nadawać im priorytety w zależności od ich ważności. 

Aplikacja wykorzystuje bazę danych `ROOM` do przechowywania wszystkich zadań, umożliwia podstawowe operacje *CRUD* (*Create-Remove-Update-Delete*), oraz ich filtrowanie i sortowanie.

<table><tr><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExcnFqOGtlZmZkZXc0Mzg2OTJnbHYxcTZhNGZ5M29xMGV0bjNpODRveiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/c56sRiyAvvGStsM7aA/giphy.gif" width="200" /></td><td><img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExOGQzd25qaDJmcWtoZXJzcmoxNDk3NnhjOWU4dnNyNG5lemJwdGtqaiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/jS5GjdH5YAFhKu2w7l/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExdmN0aTdnZDBlaWRpMjJncXRpN253azZsM2syNWZyZjVsZ3I5NXp5MSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/tYfIxGuV0ox7rDXIMM/giphy.gif" width="200" /></td></tr></table>

- Aplikacja zaimplementowana zgodnie ze wzoccem MVVM wraz z repozytorium
- `StateFlow` został wykorzystany do łatwego zarządzania bieżącym stanem interfejsu użytkownika
- Baza danych `Room` została wykorzystana do przechowywania zadań, oraz jako *SSOT* (*Single Source Of Truth*)
- Aplikacja zawiera trzy ekrany 
    - na głównym ekranie wyświetla listę zadań, umożliwia usunięcie, oznaczenie zadania jako wykonanego, oraz szybką zmianę nadanego priorytetu
    - ekran dodania zadania
    - ekran aktualizacji zadania, który umożliwia również usunięcie
- Aplikacja wykorzystuje `Compose Navigation` w celu utworzenia nawigacji w aplikacji
- `ViewModel` został umieszczony w komponencie `Navigation` w celu uzyskania współdzielenia jego instancji przez wszystkie ekrany


Dodajmy wymagane zależności do projektu

```kotlin
    implementation ("androidx.lifecycle:lifecycle-viewmodel-compose:2.6.1")
    implementation ("androidx.lifecycle:lifecycle-viewmodel-ktx:2.6.1")
    implementation ("androidx.lifecycle:lifecycle-runtime-compose:2.6.1")

    implementation ("androidx.room:room-runtime:2.5.2")
    annotationProcessor ("androidx.room:room-compiler:2.5.2")
    kapt ("androidx.room:room-compiler:2.5.2")
    implementation("androidx.room:room-ktx:2.5.2")

    implementation ("androidx.navigation:navigation-compose:2.6.0")

    implementation ("androidx.compose.material:material-icons-extended:1.4.3")
```

W pierwszym kroku przygotujmy dane, które wykorzystamy jako *dummy data* do inicjalizacji bazy danych.

In [ ]:
data class Task(val title: String, val description: String)

In [ ]:
object DataProvider {

    private val titles = listOf(
        "Dokończ Raport",
        "Przygotuj Prezentację",
        "Zadzwoń do Klienta",
        "Aktualizuj Zawartość Strony",
        "Weź Udział w Spotkaniu Zespołu",
        "Przejrzyj Kod",
        "Wysyłka Faktur",
        "Badanie Trendów na Rynku",
        "Plan Kampanii Marketingowej",
        "Organizuj Pliki",
        "Przygotuj Propozycję Budżetu",
        "Twórz Mockupy Produktu",
        "Planuj Posty na Mediach Społecznościowych",
        "Spotkanie z Zespołem Projektowym",
        "Testuj Nowe Oprogramowanie",
        "Dochodź do Potencjalnych Klientów",
        "Projektuj Logo",
        "Napisz Post na Blogu",
        "Ustal Harmonogram Projektu",
        "Przejrzyj Specyfikacje Produktu",
        "Organizuj Plan Podróży",
        "Przeprowadź Ankietę Użytkowników",
        "Analizuj Dane Sprzedażowe",
        "Przygotuj Materiały Szkoleniowe",
        "Koordynuj Logistykę Wydarzenia",
        "Burza Mózgów nad Nowymi Pomysłami",
        "Optymalizuj SEO Strony",
        "Przeprowadź Oceny Wyników Pracowników",
        "Twórz Prototypy Aplikacji"
    )

    private val descriptions = listOf(
        "Ukończ końcowy raport dotyczący wyników kwartalnych.",
        "Stwórz prezentację PowerPoint na nadchodzące spotkanie z klientem.",
        "Skontaktuj się z klientem w celu dalszej rozmowy.",
        "Aktualizuj treść na stronie głównej witryny internetowej.",
        "Zorganizuj logistykę nadchodzącego wydarzenia firmowego.",
        "Burz mózg nad innowacyjnymi pomysłami na poprawę produktu.",
        "Optymalizuj SEO strony internetowej dla lepszych wyników w wyszukiwarkach.",
        "Przeprowadź oceny wyników pracowników zespołu.",
        "Twórz interaktywne prototypy dla nowej aplikacji.",
        "Dokładnie przetestuj i oceniaj funkcjonalność nowego oprogramowania. Dokumentuj wszelkie błędy lub problemy i współpracuj blisko z zespołem developerskim, aby zapewnić ich szybkie rozwiązanie.",
        "Kontynuuj kontakt z potencjalnymi klientami i odpowiedz na zapytania wygenerowane przez kampanie marketingowe i wydarzenia networkingowe. Udziel spersonalizowanych odpowiedzi i dostarcz informacji dostosowanych do potrzeb każdego potencjalnego klienta.",
        "Zaprojektuj nowe logo dla przemiany marki, biorąc pod uwagę tożsamość marki, grupę docelową i trendy w branży. Przedstaw wiele opcji logo w celu uzyskania opinii i wybierz ostateczny projekt.",
        "Napisz dobrze przemyślany i informacyjny post na blogu na temat najnowszych wydarzeń w branży. Wykorzystaj dane, statystyki i wiedzę ekspertów, aby dostarczyć cenne treści czytelnikom bloga.",
        "Ustal harmonogram dla nadchodzących faz projektu, uwzględniając dostępność zasobów, zależności i potencjalne ryzyka. Podziel się harmonogramem z interesariuszami projektu w celu uzyskania opinii.",
        "Przejrzyj i sfinalizuj specyfikacje produktu, upewniając się, że są one kompleksowe i zgodne z celami projektu. Podziel się specyfikacjami z zespołem developerskim w celu wdrożenia.",
        "Zaplanuj plan podróży służbowej, uwzględniając rozkłady lotów, zakwaterowanie, transport i organizację spotkań. Podziel się planem podróży ze wszystkimi odpowiednimi członkami zespołu.",
        "Przeprowadź szczegółową ankietę, aby pozyskać opinię użytkowników na temat użyteczności produktu, funkcji i ogólnego zadowolenia. Analizuj wyniki ankiety, aby zidentyfikować obszary do poprawy.",
        "Analizuj dane sprzedażowe i identyfikuj trendy w celu podejmowania decyzji opartych na danych dotyczących strategii sprzedaży i poprawek produktowych. Przygotuj obszerny raport, aby przedstawić wyniki zespołowi sprzedażowemu i zarządowi.",
        "Przygotuj materiały szkoleniowe dla nowych pracowników, w tym przewodniki wprowadzające, moduły szkoleniowe i prezentacje. Upewnij się, że materiały obejmują wszystkie istotne aspekty roli nowego pracownika.",
        "Koordynuj logistykę nadchodzącego wydarzenia firmowego, takiego jak konferencja czy wyjazd integracyjny zespołu. Zabezpiecz lokalizacje, catering, transport i wszelkie niezbędne wyposażenie.",
        "Burz mózg nad innowacyjnymi pomysłami na poprawę produktu, uwzględniając opinię użytkowników, trendy rynkowe i postęp technologiczny. Przedstaw pomysły zespołowi rozwoju produktu w celu oceny.",
        "Optymalizuj SEO strony internetowej, aby poprawić pozycje w wynikach wyszukiwania i zwiększyć ruch organiczny. Przeprowadź badania słów kluczowych, aktualizuj meta tagi i wdroż na stronie najlepsze praktyki SEO.",
        "Przeprowadź oceny wyników pracowników, dostarczając konstruktywną opinię i wyznaczając cele rozwoju osobistego i zawodowego. Doceniaj i nagradzaj wyjątkowe osiągnięcia.",
        "Twórz interaktywne prototypy dla nowej aplikacji, uwzględniając przyjazny dla użytkownika design i intuicyjną nawigację. Testuj prototypy z potencjalnymi użytkownikami, aby uzyskać opinię i walidację."
    )

    val tasks = (0..4).map { Task(
        title = titles.random(),
        description = descriptions.random(),
        priority = Priority.values().random()
    ) }
}

## Baza danych

Model zadania zawiera pola
-  `title: String` - tytuł zadania
-  `description: String` - opis zadania
-  `isDone: Boolean = false` - flaga określająca czy zadanie zostało wykonane
-  `priority: Priority = Priority.NORMALNY` - priorytet zadania

In [ ]:
@Entity(tableName = "task_table")
data class Task(
    @PrimaryKey(autoGenerate = true) val id: Int = 0,
    val title: String,
    val description: String,
    val isDone: Boolean = false,
    val priority: Priority = Priority.NORMALNY
)

Priorytet zdefiniujemy w klasie `enum`

In [ ]:
enum class Priority {
    WYSOKI,
    NORMALNY,
    NISKI,
    WYKONANY
}

Dodajmy interfejs `dao` zawierający definicje operacji na bazie danych. 

In [ ]:
@Dao
interface TaskDao {

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    suspend fun insertTask(task: Task)

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    suspend fun insertAllTasks(tasks: List<Task>)

    @Query("SELECT * FROM task_table ORDER BY id ASC")
    fun getTasks(): Flow<List<Task>>

    @Delete
    suspend fun deleteTask(task: Task)

    @Update
    suspend fun updateTask(task: Task)

    @Query("DELETE FROM task_table")
    suspend fun deleteAll()
}

Dodajmy repozytorium z odpowiednimi metodami.

In [ ]:
class TaskRepository(private val application: Application) {
    private val db = TaskDatabase.getDatabase(application)
    private val dao = db.taskDao()

    fun getTasks() = dao.getTasks()
    suspend fun insertTask(task: Task) = dao.insertTask(task)
    suspend fun insertAllTasks(tasks: List<Task>) = dao.insertAllTasks(tasks)
    suspend fun deleteTask(task: Task) = dao.deleteTask(task)
    suspend fun updateTask(task: Task) = dao.updateTask(task)
    suspend fun deleteAll() = dao.deleteAll()
}

Następnie dodajmy `ViewModel`, który zarządza danymi związanymi z zadaniami, interakcjami z repozytorium (dostęp do danych) oraz logiką biznesową związaną z priorytetami i stanem zadań.

- tasksState - `StateFlow` z listą zadań. `stateIn` używa `viewModelScope` do obsługi cyklu życia i definiuje, jak strumień powinien być obsługiwany.
- Inicjalizacja bazy danych z danymi przykładowymi - w metodzie `init`, `reinitializeDatabaseWithDummyData()` jest wywołane w celu dodania przykładowych danych do bazy danych poprzez usuwanie istniejących zadań i dodawanie nowych, pochodzących z `DataProvider.tasks`.
- Operacje CRUD na bazie danych - metody `addAll`, `deleteAll`, `updateTask`, `insertTask` i `deleteTask` są wykorzystywane do interakcji z bazą danych poprzez repozytorium. Wszystkie działania są wykonywane wewnątrz bloku `viewModelScope.launch`, co oznacza, że są asynchroniczne i bezpieczne w kontekście wątków.
- Logika priorytetów i stanu zadań - pozostała część kodu zawiera logikę związaną z priorytetami i stanem zadań. Metody `getTaskColor`, `increasePriority`, `decreasePriority` i `donePriority` zmieniają priorytet oraz stan zadań w zależności od ich obecnych wartości.
- Funkcje pomocnicze - `getTask(id: Int)` pozwala na pobranie konkretnego zadania o podanym `id`. `getTaskColor(task: Task)` zwraca kolor w zależności od priorytetu zadania.

In [ ]:
class TaskViewModelFactory(private val application: Application) :
    ViewModelProvider.Factory {
    override fun <T : ViewModel> create(modelClass: Class<T>): T {
        return TaskViewModel(application) as T
    }
}

In [ ]:
class TaskViewModel(application: Application) : ViewModel() {

    private val repository = TaskRepository(application)

    val tasksState: StateFlow<List<Task>> = repository.getTasks().stateIn(
        viewModelScope,
        SharingStarted.WhileSubscribed(),
        emptyList()
    )

    init {
        reinitializeDatabaseWithDummyData()
    }

    private fun reinitializeDatabaseWithDummyData(){
        deleteAll()
        addAll(DataProvider.tasks)
    }

    private fun addAll(tasks: List<Task>){
        viewModelScope.launch {
            repository.insertAllTasks(tasks)
        }
    }

    private fun deleteAll(){
        viewModelScope.launch {
            repository.deleteAll()
        }
    }

    fun updateTask(task: Task){
        viewModelScope.launch {
            repository.updateTask(task)
        }
    }

    fun getTask(id: Int) =
        tasksState.value.find { it.id == id } ?: Task(title = "", description = "")

    fun addTask(task: Task){
        viewModelScope.launch {
            repository.insertTask(task)
        }
    }

    fun deleteTask(task: Task){
        viewModelScope.launch{
            repository.deleteTask(task)
        }
    }

    fun getTaskColor(task: Task): Color{
        return when(task.priority) {
            Priority.WYSOKI -> Color.Red
            Priority.NISKI -> Color.Blue
            Priority.NORMALNY -> Color.Green
            else -> Color(128, 203, 196)
        }
    }

    fun increasePriority(task: Task) {
        val nextPriority = when (task.priority) {
            Priority.WYSOKI -> Priority.WYSOKI
            Priority.NORMALNY -> Priority.WYSOKI
            Priority.NISKI -> Priority.NORMALNY
            Priority.WYKONANY -> Priority.WYKONANY
        }

        updateTask(task.copy(priority = nextPriority))
    }

    fun decreasePriority(task: Task) {
        val nextPriority = when (task.priority) {
            Priority.WYSOKI -> Priority.NORMALNY
            Priority.NORMALNY -> Priority.NISKI
            Priority.NISKI -> Priority.NISKI
            Priority.WYKONANY -> Priority.WYKONANY
        }

        updateTask(task.copy(priority = nextPriority))
    }

    fun donePriority(task: Task) {
        val nextPriority = when (task.priority) {
            Priority.WYSOKI -> Priority.WYKONANY
            Priority.NORMALNY -> Priority.WYKONANY
            Priority.NISKI -> Priority.WYKONANY
            Priority.WYKONANY -> Priority.WYKONANY
        }

        updateTask(task.copy(priority = nextPriority, isDone = true))
    }
}

Aplikacja posiada trzy ekrany, głównym jest lista wszystkich zadań. W nawigacji przejdziemy bezargumentowo na ekran dodania nowego zadania. Ekran aktualizacji zadania będzie wymagał przekazania `id` zadania, dzięki któremu załadujemy odpowiednie dane z listy. Sam `ViewModel` będzie **współdzielony** przez wszystkie ekrany, osiągniemy to poprzez jego zainicjowanie w komponencie `Navigation` i przekazanie tej instancji przez parametr do komponentów reprezentujących ekrany.

## Nawigacja

Dodajmy nawigację.

In [ ]:
sealed class Screens(
    val route: String
) {
    object TaskList : Screens("tasklist")
    object Add : Screens("add")
    object Update : Screens("update")
}

In [ ]:
@Composable
fun Navigation() {
    val navController = rememberNavController()
    val viewModel: TaskViewModel = viewModel(
        LocalViewModelStoreOwner.current!!,
        "TaskViewModel",
        TaskViewModelFactory(LocalContext.current.applicationContext as Application)
    )

    NavHost(navController = navController, startDestination = Screens.TaskList.route) {
        composable(route = Screens.TaskList.route){
            TaskListScreen (
                navController = navController,
                viewModel = viewModel
            )
        }

        composable(route = Screens.Add.route){
            AddTaskScreen(
                onHome = { navController.popBackStack() },
                viewModel = viewModel
            )
        }

        composable(route = Screens.Update.route + "/{arg}"){
            val arg = it.arguments?.getString("arg")
            if (arg != null)
                UpdateTaskScreen(
                    id = arg,
                    viewModel = viewModel,
                    onHome = { navController.popBackStack() }
                )
        }
    }
}

- `NavHost` - to kontener dla twoich ekranów nawigacji. Jest to miejsce, w którym zdefiniowane są możliwe ekrany do nawigacji oraz jakie komponenty są powiązane z każdym ekranem.
- `rememberNavController` - tworzy i zapamiętuje kontroler nawigacji, który pozwala na zarządzanie nawigacją między ekranami.
- `viewModel` - tworzy instancję `TaskViewModel`.
- Rozpoczęcie nawigacji z ekranami - definujemy poszczególne ekrany nawigacji za pomocą `composable`. Każdy ekran jest związany z unikalnym `route`.
- `TaskListScreen` - ekran, na którym wyświetlamy listę zadań. Przekazujemy `navController` oraz `viewModel` do tego komponentu.
- `AddTaskScreen` - ekran dodawania nowego zadania. Przekazujemy `onHome` do obsługi funkcji powrotu oraz `viewModel` do komponentu.
- `UpdateTaskScreen` - ekran aktualizacji zadania. Otrzymujemy `arg` z argumentów nawigacji, który zawiera `ID` zadania do aktualizacji. Jeśli `arg` istnieje, tworzymy `UpdateTaskScreen`, przekazując `id`, `viewModel` oraz `onHome`.

Ta funkcja jest centralnym punktem obsługi nawigacji w aplikacji.

Tą funkcję wywołujemy w aktywności głównej aplikacji.

In [ ]:
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContent {
            PrioritizeMeComposeTheme {
                // A surface container using the 'background' color from the theme
                Surface(
                    modifier = Modifier.fillMaxSize(),
                    color = MaterialTheme.colorScheme.background
                ) {
                    Navigation()
                }
            }
        }
    }
}

## Ekrany

Przejdźmy do definicji komponentu renderującego ekran z listą zadań.

Dodajmy kilka funkcji pomocniczych. Przejście na ekran dodawania nowego zadania zrealizujemy za pomocą **FAB** (*Floating Action Button*), jest to interaktywny przycisk, który znajduje się zazwyczaj na dolnym prawym rogu ekranu.

In [ ]:
@Composable
private fun AddTaskFAB(
    onClick: () -> Unit
) {
    FloatingActionButton(
        onClick = onClick,
        elevation = FloatingActionButtonDefaults.bottomAppBarFabElevation(10.dp),
        containerColor = Color(100, 149, 237),
        shape = FloatingActionButtonDefaults.smallShape
    ) {
        Icon(
            imageVector = Icons.Filled.Add,
            contentDescription = "Add item"
        )
    }
}

- `onClick = onClick` - Ustala funkcję `onClick` przekazaną jako parametr jako akcję do wykonania po kliknięciu na FAB.
- `elevation` Ustawia efekt cienia FAB, tutaj używając `FloatingActionButtonDefaults.bottomAppBarFabElevation` z wartością `10.dp`.
- `containerColor` Określa kolor tła FAB.
- `shape` Ustawia kształt FAB. Tutaj używamy `FloatingActionButtonDefaults.smallShape`, co nadaje FAB niewielki zaokrąglony kształt.
- `Icon` - To jest komponent `Compose`, który generuje ikonę wewnątrz FAB.
- `imageVector` Ustawia ikonę używając `Icons.Filled.Add`, co oznacza ikonę plusa.
- `contentDescription` Określa opis treści dla ikony, co jest ważne dla dostępności (czytniki ekranu).

Następnym komponentem będzie ikona priorytetu - priorytet na liście będzie reprezentowany przez kolor na pasku tytułu zadania.

In [ ]:
@Composable
private fun CreatePriorityIcon(color: Color) {
    Box(
        modifier = Modifier
            .fillMaxHeight()
            .padding(end = 4.dp)
    )
    {
        Card(
            elevation = CardDefaults.cardElevation(10.dp),
            colors = CardDefaults.cardColors(
                containerColor = Color(0xFF6650a4)
            )
        ) {
            Image(
                imageVector = Icons.Default.Circle,
                contentDescription = "Priority",
                colorFilter = ColorFilter.tint(color)
            )
        }
    }
}

- `Card` jest komponentem, który generuje efekt cienia wokół elementu, przypominając kartę.
- `elevation` Ustawia efekt cienia karty, tutaj używając `CardDefaults.cardElevation`.
- `colors` Ustawia kolory karty, tutaj zmieniamy kolor tła (`containerColor`).
- `Image` - jest komponentem, który generuje obrazek.
- `imageVector` Ustawia ikonę jako koło (`Icons.Default.Circle`).
- `colorFilter` Zastosowuje filtr koloru, aby zmienić kolor obrazka (`ColorFilter.tint`) zgodnie z przekazanym parametrem.

Na każdej karcie będą znajdować się przyciski umożliwiające wykonanie kilku akcji. Dodajmy funkcję renderującą pojedynczy przycisk.

In [ ]:
@Composable
private fun CreateOutlinedIconButton(
    onClick: () -> Unit,
    icon: ImageVector
) {
    OutlinedIconButton(
        onClick = onClick,
        enabled = true,
        modifier = Modifier.size(40.dp),
        border = BorderStroke(2.dp, Color(0xFF6650a4)),
        colors = IconButtonDefaults.iconButtonColors(
            contentColor = Color(0xFF6650a4),
        ),
        content = {
            Icon(
                imageVector = icon,
                contentDescription = ""
            )
        }
    )
}

- `OutlinedIconButton` - generuje przycisk z obrysowanym konturem, z ikoną w środku.
- `enabled` -  Ustala, czy przycisk jest aktywny (klikalny).
- `border` Ustala obrys konturu przycisku za pomocą `BorderStroke`.
- `colors` Ustala kolory przycisku, tutaj zmieniamy kolor treści (`contentColor`).

Dodajmy komponent renderujący wszystkie przyciski, które będą częścią każdego zadania.

In [ ]:
@Composable
private fun ButtonsRow(
    onClickHigherPriority: () -> Unit,
    onClickDone: () -> Unit,
    onClickLowerPriority: () -> Unit,
    onClickEdit: () -> Unit,
    onDelete: () -> Unit
) {
    Column {
        Row(
            modifier = Modifier
                .fillMaxWidth()
                .padding(top = 8.dp),
            Arrangement.SpaceEvenly,
            verticalAlignment = Alignment.CenterVertically
        ) {
            CreateOutlinedIconButton(
                onClick = onClickHigherPriority,
                icon = Icons.Outlined.KeyboardArrowUp
            )
            CreateOutlinedIconButton(
                onClick = onClickDone,
                icon = Icons.Outlined.Done
            )
            CreateOutlinedIconButton(
                onClick = onClickLowerPriority,
                icon = Icons.Outlined.KeyboardArrowDown
            )

            OutlinedIconButton(
                onClick = onDelete,
                shape = FloatingActionButtonDefaults.smallShape,
                modifier = Modifier.padding(start = 4.dp),
                border = BorderStroke(2.dp, Color(0xFF6650a4)),
                colors = IconButtonDefaults.iconButtonColors(
                    contentColor = Color(0xFF6650a4),
                ),
            ) {
                Icon(
                    imageVector = Icons.Filled.Delete,
                    contentDescription = "Delete",
                    tint = Color(0xAA90021F)
                )
            }
        }
        OutlinedIconButton(
            onClick = onClickEdit,
            enabled = true,
            modifier = Modifier
                .fillMaxWidth()
                .padding(start = 4.dp, end = 4.dp),
            border = BorderStroke(2.dp, Color(0xFF6650a4)),
            colors = IconButtonDefaults.iconButtonColors(
                contentColor = Color(0xFF6650a4),
            ),
            content = {
                Icon(
                    imageVector = Icons.Outlined.Edit,
                    contentDescription = "Edit task"
                )
            }
        )
    }
}

W pierwszym rzędzie znajdują się przyciski do zmiany priorytetu, ustawienia na wykonane oraz usunięcia klikniętego zadania. W drugim rzędzie znajduje się przycisk umożliwiający przejście na ekran edycji.

Każda karta reprezentująca zadanie będzie posiadała pasek tytułu zawierający sam tytuł, oraz ikonę reprezentującą aktualny priorytet. Utwórzmy komponent reprezentujący pasek tytułu.

In [ ]:
@Composable
private fun CardTitle(task: Task, viewModel: TaskViewModel) {
    Box(
        modifier = Modifier
            .fillMaxWidth()
            .background(Color(147, 176, 230))

    ) {
        Row(
            modifier = Modifier
                .fillMaxWidth(),
            verticalAlignment = Alignment.CenterVertically
        ) {
            Text(
                text = task.title,
                fontWeight = FontWeight.Bold,
                style = MaterialTheme.typography.titleMedium,
                modifier = Modifier
                    .padding(4.dp)
                    .weight(1f),
                fontSize = 16.sp,
                textAlign = TextAlign.Center,
            )

            CreatePriorityIcon(viewModel.getTaskColor(task))
        }
    }
}

Dodajmy komponent renderujący samą kartę.

In [ ]:
@Composable
private fun CreateItemCard(
    onClickEdit: () -> Unit,
    task: Task,
    viewModel: TaskViewModel
) {
    Card(
        shape = RoundedCornerShape(15.dp),
        elevation = CardDefaults.cardElevation(defaultElevation = 10.dp),
        border = BorderStroke(2.dp, Color.Blue)
    ) {
        Column(
            modifier = Modifier.fillMaxSize()
        ) {
            CardTitle(task, viewModel)                // pasek tytułu
            Text(                                     // opis zadania
                text = task.description,
                modifier = Modifier.padding(6.dp),
                fontSize = 14.sp
            )
            ButtonsRow(                               // przyciski modyfikujące zadanie
                onClickHigherPriority = { viewModel.increasePriority(task) },
                onClickDone = { viewModel.donePriority(task) },
                onClickLowerPriority = { viewModel.decreasePriority(task) },
                onClickEdit = onClickEdit,
                onDelete = { viewModel.deleteTask(task) }
            )
        }
    }
}

Ostatnim elementem będzie dodanie głównej funkcji. Podstawowym elementem ekranu będzie `Scaffold` przez który dodamy `FAB` oraz samą listę zadań, lista zostanie wygenerowana za pomocą `LazyVerticalStaggeredGrid`, który umożliwia tworzenie pionowych układów siatkowych z elementami o różnych wysokościach.

In [ ]:
@OptIn(ExperimentalFoundationApi::class, ExperimentalMaterial3Api::class)
@Composable
fun TaskListScreen(
    navController: NavHostController,
    viewModel: TaskViewModel
) {

    val tasks by viewModel.tasksState.collectAsStateWithLifecycle()

    Scaffold(
        floatingActionButton = {                                                  // Floating Action Button
            AddTaskFAB(onClick = { navController.navigate(Screens.Add.route) })
        },
    ) { paddingValues ->
        Box(
            modifier = Modifier
                .fillMaxSize()
                .padding(paddingValues)
                .padding(top = 12.dp)
        ) {
            LazyVerticalStaggeredGrid(                                            // niesymetryczna siatka elementów
                columns = StaggeredGridCells.Adaptive(200.dp),                    // każda kolumna musi posiadać szerokość co najmniej 200.dp
                verticalItemSpacing = 8.dp,                                       // odstęp między kolumnami
                horizontalArrangement = Arrangement.spacedBy(8.dp),               // odstęp między elementami
                content = {
                    items(count = tasks.size) { index ->
                        CreateItemCard(
                            onClickEdit = { navController.navigate(Screens.Update.route + "/${tasks[index].id}")},
                            task = tasks[index],
                            viewModel = viewModel
                        )
                    }
                },
                modifier = Modifier.fillMaxSize()
            )
        }
    }
}

><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExcnFqOGtlZmZkZXc0Mzg2OTJnbHYxcTZhNGZ5M29xMGV0bjNpODRveiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/c56sRiyAvvGStsM7aA/giphy.gif" width="200" />

Następnie dodajmy ekran nowego zadania.

In [ ]:
@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun AddTaskScreen(onHome: () -> Unit, viewModel: TaskViewModel){

    val radioOptions = Priority.values().toList()    // lista wszystkich priorytetów
    
    // selectedOption to aktualna wartość stanu (wybrana opcja), a onOptionSelected to funkcja, która pozwoli na zmianę tej wartości.
    val (selectedOption, onOptionSelected) = remember { mutableStateOf( radioOptions[1]) }
    val checkedState = remember { mutableStateOf(true) } // aktualny stan checkboxa
    val enabled = remember { mutableStateOf(true) }      // aktualny stan edytowalności radiobutton

    var task by remember {
        mutableStateOf(Task(title = "", description = "")) // tworzymy puste zadanie
    }
    
    Column(modifier = Modifier
        .fillMaxSize()
        .padding(8.dp)
    ) {
        OutlinedTextField(
            value = task.title,
            onValueChange = { task = task.copy(title = it) }, // przy zmianie tworzymy nowe zadanie przez wykonanie kopii
            modifier = Modifier.fillMaxWidth(),
            singleLine = true,
            label = { Text(text = "Tytuł")}
        )

        OutlinedTextField(
            value = task.description,
            onValueChange = { task = task.copy(description = it) },
            modifier = Modifier
                .fillMaxWidth()
                .fillMaxHeight(.5f),
            singleLine = false,
            label = { Text(text = "Opis")}
        )
        
        Row(
            modifier = Modifier
                .fillMaxWidth()
                .padding(top = 8.dp),
        ) {
            OutlinedCard(
                modifier = Modifier
                    .fillMaxWidth(.6f)
                    .padding(end = 2.dp),
            ) {
                Column(
                    verticalArrangement = Arrangement.SpaceEvenly,
                    horizontalAlignment = Alignment.CenterHorizontally
                ) {
                    // RadioButton z wyborem priorytetu
                    radioOptions.forEach {text ->
                        Row(
                            Modifier
                                .fillMaxWidth()
                                .selectable(
                                    selected = (text == selectedOption),
                                    onClick = { onOptionSelected(text) },
                                    enabled = enabled.value
                                )
                                .padding(horizontal = 16.dp),
                            verticalAlignment = Alignment.CenterVertically
                        ) {
                            RadioButton(
                                selected = (text == selectedOption),  // stan zaznaczenia
                                onClick = { onOptionSelected(text) }, // zmiana selectedOption
                                enabled = enabled.value               // czy przycisk jest dostępny
                            )
                            TextField(
                                value = text.toString(),
                                onValueChange = {},
                                modifier = Modifier
                                    .padding(start = 16.dp),
                                enabled = enabled.value, // dostępność pola
                                singleLine = true,
                                readOnly = true, // pole tylko do odczytu
                                textStyle = TextStyle.Default.copy(fontSize = 16.sp),
                                // usunięcie domyślnych kolorów pola
                                colors = TextFieldDefaults.textFieldColors(
                                    focusedIndicatorColor = Color.Transparent,
                                    unfocusedIndicatorColor = Color.Transparent,
                                    disabledIndicatorColor = Color.Transparent,
                                    containerColor = Color.White
                                )
                            )
                        }
                    }
                }
            }

            OutlinedCard(
                modifier = Modifier
                    .fillMaxWidth()
                    .height(144.dp)
                    .padding(start = 2.dp)
            ) {
                Column(
                    modifier = Modifier.fillMaxSize(),
                    horizontalAlignment = Alignment.CenterHorizontally,
                    verticalArrangement = Arrangement.Center
                ) {
                    Text(
                        text = "Czy zakończone",
                        fontSize = 16.sp,
                        modifier = Modifier.padding(start = 4.dp)
                    )
                    // checkbox zmieniający stan checkedState
                    Checkbox(
                        modifier = Modifier
                            .scale(1.2f)
                            .padding(top = 8.dp),
                        checked = task.isDone,
                        onCheckedChange = {
                            checkedState.value = it
                            task = task.copy(isDone = it) // przy zmianie zapisujemy aktualny stan w zadaniu
                            if (checkedState.value) {
                                enabled.value = false // zmiana dostępności radioButton - gdy zaznaczone przyciski są niedostępne
                                onOptionSelected(Priority.WYKONANY) // zmaina priorytetu na WYKONANY
                            }
                            else
                                enabled.value = true // jeżeli nie zaznaczony, umożliwiamy edycję radioButton i wybór priorytetu
                        }
                    )
                }
            }
        }
        Row(
            modifier = Modifier
                .fillMaxWidth()
                .padding(top = 8.dp)
        ) {
            Button(
                onClick = {
                    task = task.copy(priority = selectedOption) // ustawienie priorytetu zadania
                    if (task.title.isNotEmpty() && task.description.isNotEmpty()) // zapis wykonujemy gdy pola nie są puste
                        viewModel.addTask(task) // dodanie zadania do bazy danych
                    onHome() // powrót na ekran listy
                },
                modifier = Modifier.fillMaxWidth(.5f)
            ) {
                Text(
                    text = "Zapisz",
                    fontSize = 20.sp
                )
            }
            OutlinedButton(
                onClick = onHome,
                modifier = Modifier.fillMaxWidth()
            ) {
                Text(
                    text = "Wróć",
                    fontSize = 20.sp
                )
            }
        }
    }
}

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExOGQzd25qaDJmcWtoZXJzcmoxNDk3NnhjOWU4dnNyNG5lemJwdGtqaiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/jS5GjdH5YAFhKu2w7l/giphy.gif" width="200" />

Ostatnim ekranem będzie ekran aktualizacji zadania, będzie on podobny do ekranu dodawania zadania.

In [ ]:
@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun UpdateTaskScreen(id: String, viewModel: TaskViewModel, onHome: () -> Unit){

    val radioOptions = Priority.values().toList()
    val checkedState = remember { mutableStateOf(true) }


    var task by remember {
        mutableStateOf(viewModel.getTask(id.toInt())) // wyszukanie zadania o id przekazanym w parametrze
    }
    val (selectedOption, onOptionSelected) = remember { mutableStateOf( task.priority) }
    val enabled = remember { mutableStateOf(!task.isDone) }
    
    Column(modifier = Modifier
        .fillMaxSize()
        .padding(8.dp)
    ) {
        OutlinedTextField(
            value = task.title,
            onValueChange = {task = task.copy(title = it)},
            modifier = Modifier.fillMaxWidth(),
            singleLine = true,
            label = { Text(text = "Tytuł")}
        )

        OutlinedTextField(
            value = task.description,
            onValueChange = {task = task.copy(description = it)},
            modifier = Modifier
                .fillMaxWidth()
                .fillMaxHeight(.4f),
            singleLine = false,
            label = { Text(text = "Opis")}
        )
        
        Row(
            modifier = Modifier
                .fillMaxWidth()
                .padding(top = 8.dp),
        ) {
            OutlinedCard(
                modifier = Modifier
                    .fillMaxWidth(.6f)
                    .padding(end = 2.dp),
            ) {
                Column(
                    verticalArrangement = Arrangement.SpaceEvenly,
                    horizontalAlignment = Alignment.CenterHorizontally
                ) {
                    radioOptions.forEach { text ->
                        Row(
                            Modifier
                                .fillMaxWidth()
                                .selectable(
                                    selected = (text == selectedOption),
                                    onClick = { onOptionSelected(text) },
                                    enabled = enabled.value
                                )
                                .padding(horizontal = 4.dp),
                            verticalAlignment = Alignment.CenterVertically
                        ) {
                            RadioButton(
                                selected = (text == selectedOption),
                                onClick = { onOptionSelected(text) },
                                enabled = enabled.value
                            )
                            TextField(
                                value = text.toString(),
                                onValueChange = {},
                                modifier = Modifier.padding(start = 2.dp),
                                enabled = enabled.value,
                                singleLine = true,
                                readOnly = true,
                                textStyle = TextStyle.Default.copy(fontSize = 16.sp),
                                colors = TextFieldDefaults.textFieldColors(
                                    focusedIndicatorColor = Color.Transparent,
                                    unfocusedIndicatorColor = Color.Transparent,
                                    disabledIndicatorColor = Color.Transparent,
                                    containerColor = Color.White
                                )
                            )
                        }
                    }
                }
            }

            OutlinedCard(
                modifier = Modifier
                    .fillMaxWidth()
                    .height(144.dp)
                    .padding(start = 2.dp)
            ) {
                Column(
                    modifier = Modifier.fillMaxSize(),
                    horizontalAlignment = Alignment.CenterHorizontally,
                    verticalArrangement = Arrangement.Center
                ) {
                    Text(
                        text = "Czy zakończone",
                        fontSize = 16.sp,
                        modifier = Modifier.padding(start = 4.dp)
                    )

                    Checkbox(
                        modifier = Modifier
                            .scale(1.2f)
                            .padding(top = 8.dp),
                        checked = task.isDone,
                        onCheckedChange = {
                            checkedState.value = it
                            task = task.copy(isDone = it)
                            if (checkedState.value) {
                                enabled.value = false
                                onOptionSelected(Priority.WYKONANY)
                            }
                            else
                                enabled.value = true
                        }
                    )
                }
            }
        }
        Row(
            modifier = Modifier
                .fillMaxWidth()
                .padding(top = 8.dp)
        ) {


            Button(
                onClick = {
                    task = task.copy(priority = Priority.valueOf(selectedOption.name))
                    viewModel.updateTask(task)
                    onHome()
                          },
                modifier = Modifier.fillMaxWidth(.5f).padding(end = 2.dp)
            ) {
                Text(
                    text = "Aktualizuj",
                    fontSize = 20.sp
                )
            }
            Button(
                onClick = {
                    viewModel.deleteTask(task) // usunięcie zadania
                    onHome()
                          },
                modifier = Modifier.fillMaxWidth().padding(start = 2.dp)
            ) {
                Text(
                    text = "Usuń",
                    fontSize = 20.sp
                )
            }
        }
        OutlinedButton(
            onClick = onHome,
            modifier = Modifier.fillMaxWidth()
        ) {
            Text(
                text = "Wróć",
                fontSize = 20.sp
            )
        }
    }
}

<img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExdmN0aTdnZDBlaWRpMjJncXRpN253azZsM2syNWZyZjVsZ3I5NXp5MSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/tYfIxGuV0ox7rDXIMM/giphy.gif" width="200" />